# THE PLAN

## 1. ask questions
#### Two for Naomi, Two for Woody

## 2. Create Visualizations
#### 5 ugly to answer the questions: Woody
#### 2 nice viz to answer Naomi's questions: Naomi
#### 2 nice Viz to answer Woody's questions: Woody
#### 1 nice viz for general data: Naomi

## 3. Run Stat Test 
#### 1 stat test to answer Woody question: Woody
#### 1 stat test to answer Naomi question: Naomi

## 4. Wrap it all in Markdown
### Naomi's whole job

## QUESTIONS: 
#### Naomi's human eyeballs question : how does clustering valuable out-door features arrange the data? Is the data evenly distributed between the clusters?
#### Naomi's computer brain question: Does the clustering algorithnm identify features that consistnely have logerror close to zero?

#### Woody question 1: Does clustering the cluster of tax_value and calc_sqft have a closer RMSE to logerror than baseline?
#### Woody question 2: ?????



# Welcome to our Project!

### Goal of This Project: 
- To predict logerror